In [25]:
import pandas as pd
import spacy
import sqlite3
from sqlalchemy import create_engine

from spacytextblob.spacytextblob import SpacyTextBlob

from goodreads_book_scraper import get_listopia
from book_importer import BookImporter
from language_analysis import Book
from similarity import SimilarityChecker


# Data Pipeline

## Get a list of books from GoodReads

In [26]:

# Get the URL from the GoodReads website
best_books_ever_url = 'https://www.goodreads.com/list/show/1.Best_Books_Ever'
large_book_list_url = 'https://www.goodreads.com/list/show/952.1001_Books_You_Must_Read_Before_You_Die'
this_week_url = 'https://www.goodreads.com/book/most_read'

# Scrape the website and return lists of books, authors and average review
goodreads_list, author_list, review_list = get_listopia(this_week_url)



In [27]:
# Empty book_data list to store the books in
book_data = []

# Iterate through the book list and check if it is on Gutenberg
for count, book in enumerate(goodreads_list):
    file_path, book_id = BookImporter.gutendex(book, author_list[count])
    
    # Store the book in the book list
    book_data.append({'GutenbergID': book_id, 'Title': book, 'Author': author_list[count], 'Review': review_list[count], 'FilePath': file_path})


In [28]:
# Convert the book list into a pandas and check for missing values
book_data = pd.DataFrame(book_data)
book_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   GutenbergID  100 non-null    int64 
 1   Title        100 non-null    object
 2   Author       100 non-null    object
 3   Review       100 non-null    object
 4   FilePath     21 non-null     object
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


### Remove Duplicates (if any)

In [29]:
# Check for duplicates
duplicates = book_data.duplicated()

duplicate_rows = book_data[duplicates]
print(duplicate_rows)

    GutenbergID                  Title      Author Review FilePath
93            0  To Kill a Mockingbird  Harper Lee   4.27     None


In [30]:
# If there are any duplicate values (check by title and author), remove them
book_data = book_data.drop_duplicates(subset=['Title', 'Author'], keep='first')

In [31]:
# Check the dataframe
print(book_data.head())

   GutenbergID                                              Title   
0            0            The Hunger Games (The Hunger Games, #1)  \
1            0  Harry Potter and the Order of the Phoenix (Har...   
2         1342                                Pride and Prejudice   
3            0                              To Kill a Mockingbird   
4            0                                     The Book Thief   

            Author Review                      FilePath  
0  Suzanne Collins   4.33                          None  
1     J.K. Rowling   4.50                          None  
2      Jane Austen   4.28  data/Pride_and_Prejudice.txt  
3       Harper Lee   4.27                          None  
4     Markus Zusak   4.39                          None  


In [32]:
# Get a list of unique author names
author_list = book_data['Author'].unique()
author_list_df = pd.DataFrame({'Name': author_list})

print(author_list_df)

                            Name
0                Suzanne Collins
1                   J.K. Rowling
2                    Jane Austen
3                     Harper Lee
4                   Markus Zusak
..                           ...
84               Edgar Allan Poe
85            Barbara Kingsolver
86                     Anne Rice
87  Miguel de Cervantes Saavedra
88              Ernest Hemingway

[89 rows x 1 columns]


## Add data to SQL database

### Start up SQL

In [33]:
# Start the SQL database 
conn = sqlite3.connect('databases/book_repository.db')
cursor = conn.cursor()
engine = create_engine('sqlite:///databases/book_repository.db')


### Create the tables (Commented out after the first iteration)

In [34]:
# Create Authors table
with engine.begin() as connection:
    conn.execute('''
        CREATE TABLE IF NOT EXISTS Authors(
            AuthorID INTEGER PRIMARY KEY AUTOINCREMENT,
            Name VARCHAR
        )
    ''')

In [35]:
# Create Gutenberg table
with engine.begin() as connection:
    conn.execute('''
    CREATE TABLE IF NOT EXISTS Gutenberg(
        GutenbergID INTEGER PRIMARY KEY,
        FilePath VARCHAR
        );
    ''')
        # If there is not GutenbergID, it will reference this
    conn.execute('''
        INSERT OR IGNORE INTO Gutenberg (GutenbergID, FilePath)
        VALUES (0, 'Not Available')
        ''')

In [36]:
# Create Books table
with engine.begin() as connection:
    conn.execute('''
    CREATE TABLE IF NOT EXISTS Books(
        GutenbergID VARCHAR(7),
        Title VARCHAR,
        AuthorID INTEGER,
        Review DOUBLE,
        FOREIGN KEY (GutenbergID) REFERENCES Gutenberg(GutenbergID)
        FOREIGN KEY (AuthorID) REFERENCES Authors(AuthorID)
        );
''')


### Put the data in a temporary table

In [37]:
# Turn the books data into a temporary SQL table
book_data.to_sql('temp_books', conn, if_exists='replace', index=False)

99

In [38]:
# Turn the author list into a temporary SQL table
author_list_df.to_sql('temp_authors', conn, if_exists='replace', index=False)

89

In [39]:
# Insert the author names into the table, ignore repetitions
with engine.begin() as connection:
    conn.execute("""
        INSERT INTO Authors (Name)
        SELECT DISTINCT LOWER(t.Name)
        FROM temp_authors AS t
        LEFT JOIN Authors AS a ON LOWER(t.Name) = LOWER(a.Name)
        WHERE a.Name IS NULL
    """)

In [40]:
with engine.begin() as connection:
    conn.execute('''
        INSERT INTO Gutenberg (GutenbergID, FilePath)
        SELECT t.GutenbergID, t.FilePath
        FROM temp_books as t
        LEFT JOIN Gutenberg AS g ON t.GutenbergID = g.GutenbergID
        WHERE t.FilePath IS NOT NULL
          AND (g.GutenbergID IS NULL OR g.GutenbergID = 0)
    ''')

In [41]:
# Insert the rest of the Book data, with the foreign keys
with engine.begin() as connection:
    conn.execute('''
        INSERT INTO Books (GutenbergID, Title, AuthorID, Review)
        SELECT t.GutenbergID, t.Title, a.AuthorID, t.Review
        FROM temp_books AS t
        LEFT JOIN Authors AS a ON LOWER(t.Author) = LOWER(a.Name)
        LEFT JOIN Books AS b ON t.GutenbergID = b.GutenbergID
        WHERE b.GutenbergID IS NULL
    ''')

### Drop the temporary Table

In [42]:
# Drop the temporary tables
with engine.begin() as connection:
    conn.execute("DROP TABLE IF EXISTS temp_books")
    conn.execute('DROP TABLE IF EXISTS temp_authors')


In [43]:
# Check to see if the Data was inserted correctly
cursor.execute('''
    SELECT Books.Title, Authors.Name, Books.GutenbergID
    FROM Books
    LEFT OUTER JOIN Authors ON Books.AuthorID = Authors.AuthorID
''')

rows = cursor.fetchall()
for row in rows:
    print(f'{row[0]}: {row[1]}, {row[2]}')
print(len(rows))


The Hunger Games (The Hunger Games, #1): suzanne collins, 0
Harry Potter and the Order of the Phoenix (Harry Potter, #5): j.k. rowling, 0
Pride and Prejudice: jane austen, 1342
To Kill a Mockingbird: harper lee, 0
The Book Thief: markus zusak, 0
Twilight (The Twilight Saga, #1): stephenie meyer, 0
Animal Farm: george orwell, 0
J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings: j.r.r. tolkien, 0
The Chronicles of Narnia (Chronicles of Narnia, #1-7): c.s. lewis, 0
The Fault in Our Stars: john green, 0
Gone with the Wind: margaret mitchell, 0
The Giving Tree: shel silverstein, 0
The Picture of Dorian Gray: oscar wilde, 174
Wuthering Heights: emily brontë, 768
Harry Potter and the Philosopher’s Stone (Harry Potter, #1): j.k. rowling, 0
The Perks of Being a Wallflower: stephen chbosky, 0
Jane Eyre: charlotte brontë, 1260
The Da Vinci Code (Robert Langdon, #2): dan brown, 0
The Great Gatsby: f. scott fitzgerald, 64317
The Little Prince: antoine de saint-exupéry, 0
Alice's

In [44]:
# Check if the Gutenberg table was inserted correctly
cursor.execute('SELECT * FROM Gutenberg')

rows = cursor.fetchall()
for row in rows:
    print(f'{row[0]:>7}: {row[1]}')
print(len(rows))

      0: Not Available
     15: data/Moby-Dick_or,_the_Whale.txt
     39: data/The_Hitchhiker's_Guide_to_the_Galaxy_(The_Hitchhiker's_Guide_to_the_Galaxy,_#1).txt
     43: data/Dr._Jekyll_and_Mr._Hyde.txt
     46: data/A_Christmas_Carol.txt
     98: data/A_Tale_of_Two_Cities.txt
    105: data/Persuasion.txt
    110: data/Tess_of_the_D'Urbervilles.txt
    120: data/Treasure_Island.txt
    135: data/Les_Misérables.txt
    141: data/Mansfield_Park.txt
    158: data/Emma.txt
    161: data/Sense_and_Sensibility.txt
    174: data/The_Picture_of_Dorian_Gray.txt
    203: data/Uncle_Tom's_Cabin.txt
    215: data/The_Call_of_the_Wild.txt
    345: data/Dracula.txt
    521: data/Robinson_Crusoe.txt
    730: data/Oliver_Twist.txt
    768: data/Wuthering_Heights.txt
    996: data/Don_Quixote.txt
   1184: data/The_Count_of_Monte_Cristo.txt
   1260: data/Jane_Eyre.txt
   1342: data/Pride_and_Prejudice.txt
   1399: data/Anna_Karenina.txt
   1400: data/Great_Expectations.txt
   1513: data/Romeo_and_Juli

In [45]:
cursor.execute('SELECT * FROM Gutenberg WHERE GutenbergID=0')
print(cursor.fetchall())

[(0, 'Not Available')]


## Commit any unsaved changes

In [46]:
conn.commit()

In [47]:
# Stop the SQL server
conn.close()

# Analysis

## Load the NLP and add TextBlob to it

In [ ]:
# Load the large dataset and add the TextBlob pipeline
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe('spacytextblob')

# Create a books list to store the Book objects
books = []

## Read the files and analyse the novels

### Jekyll and Hyde

In [ ]:
# Import Jekyll and Hyde from Gutenberg
BookImporter.get_book(43, 'Jekyll and Hyde')

# Read the txt file and create a Book object
with open('data\Jekyll_and_Hyde.txt', 'r', encoding='UTF-8') as file:
    jekyll_hyde = Book('Jekyll and Hyde', file.read())

# Denote the chapter markers in a regex expression
chapter_markers = r'((\nSTORY OF THE DOOR)|(\nSEARCH FOR MR. HYDE)|(\nDR. JEKYLL WAS QUITE AT EASE)|(\nTHE CAREW MURDER CASE)|(\nINCIDENT OF THE LETTER)|(\nINCIDENT OF DR. LANYON)|(\nINCIDENT AT THE WINDOW)|(\nTHE LAST NIGHT)|(\nDR. LANYON’S NARRATIVE)|(\nHENRY JEKYLL’S FULL STATEMENT OF THE CASE))'


In [ ]:
# Split the book into chapters
jekyll_hyde.split_into_chapters(chapter_markers)


In [ ]:
# Conduct NLP analysis sentence by sentence
jekyll_hyde.do_nlp(nlp)


In [ ]:
# Do NLP on each chapter
jekyll_hyde.chapter_nlp(nlp)


In [ ]:
# Show sentiment analysis of the entire text
jekyll_hyde.blobify()

In [ ]:
# Show the analysis of the book
jekyll_hyde.get_analysis()

In [ ]:
# Show the sentiment analysis by chapter
jekyll_hyde.chapter_analysis()


In [ ]:
# Add the book to the book object list
books.append(jekyll_hyde)

### Dracula

In [ ]:
with open('data\Dracula.txt', 'r', encoding='UTF-8') as file:
    dracula = Book('Dracula', file.read())
chapter_markers = r'(PREFACE.)|(LETTER I+\.)|(CHAPTER [IVXLCDM]+\n)'



In [ ]:
dracula.split_into_chapters(chapter_markers)


In [ ]:
dracula.do_nlp(nlp)


In [ ]:
dracula.chapter_nlp(nlp)


In [ ]:
dracula.blobify()

In [ ]:
dracula.get_analysis()


In [ ]:
dracula.chapter_analysis()
books.append(dracula)

### Frankenstein

In [ ]:
BookImporter.get_book(84, 'Frankenstein')
with open('data\Frankenstein.txt', 'r', encoding='UTF-8') as file:
    frankenstein = Book('Frankenstein', file.read())

chapter_markers = r'\n((Letter .)|(Chapter .+))'
frankenstein.split_into_chapters(chapter_markers)


In [ ]:
frankenstein.do_nlp(nlp)


In [ ]:
frankenstein.chapter_nlp(nlp)


In [ ]:
frankenstein.blobify()


In [ ]:
frankenstein.get_analysis()


In [ ]:
frankenstein.chapter_analysis()
books.append(frankenstein)

### The Turn of the Screw

In [ ]:
BookImporter.get_book(209, 'The Turn of the Screw')
with open('data\The_Turn_of_the_Screw.txt', 'r', encoding='UTF-8') as file:
    turn_of_the_screw = Book('The Turn of the Screw', file.read())

chapter_markers = r'((\nI\n)|(\nII)|(\nIII)|(\nIV)|(\nV)|(\nVI)|(\nVII)|(\nVIII)|(\nIX)|(\nX)|(\nXI)|(\nXII)|(\nXIII)|(\nXIV)|(\nXV)|(\nXVI)|(\nXVII)|(\nXVIII)|(\nXIX)|(\nXX)|(\nXXI)|(\nXXII)|(\nXXIII)|(\nXXIV))'


In [ ]:
turn_of_the_screw.split_into_chapters(chapter_markers)


In [ ]:
turn_of_the_screw.do_nlp(nlp)


In [ ]:
turn_of_the_screw.chapter_nlp(nlp)


In [ ]:
turn_of_the_screw.blobify()


In [ ]:
turn_of_the_screw.get_analysis()


In [ ]:
turn_of_the_screw.chapter_analysis()
books.append(turn_of_the_screw)

### Romeo and Juliet

In [ ]:
BookImporter.get_book(1513, 'Romeo and Juliet')
with open('data\Romeo_and_Juliet.txt', 'r', encoding='UTF-8') as file:
    romeo_and_juliet = Book('Romeo and Juliet', file.read())

chapter_markers = r'(THE PROLOGUE\n)|(SCENE I. A public place)|(SCENE II. A Street)|(SCENE III. Room in Capulet’s House)|(SCENE IV. A Street)|(SCENE V. A Hall in Capulet’s House)|(ACT II\n\n)|(SCENE I. An open place adjoining Capulet’s Garden)|(SCENE II. Capulet’s Garden)|(SCENE III. Friar Lawrence’s Cell)|(SCENE IV. A Street)|(SCENE V. Capulet’s Garden)|(SCENE VI. Friar Lawrence’s Cell)|(SCENE I. A public Place)|(SCENE II. A Room in Capulet’s House)|(SCENE III. Friar Lawrence’s cell)|(SCENE IV. A Room in Capulet’s House)|(SCENE V. An open Gallery to Juliet’s Chamber, overlooking the Garden)|(SCENE I. Friar Lawrence’s Cell)|(SCENE II. Hall in Capulet’s House)|(SCENE III. Juliet’s Chamber)|(SCENE IV. Hall in Capulet’s House)|(SCENE V. Juliet’s Chamber; Juliet on the bed)|(SCENE I. Mantua. A Street)|(SCENE II. Friar Lawrence’s Cell)|(SCENE III. A churchyard; in it a Monument belonging to the Capulets)'


In [ ]:
romeo_and_juliet.split_into_chapters(chapter_markers)


In [ ]:
romeo_and_juliet.do_nlp(nlp)


In [ ]:
romeo_and_juliet.chapter_nlp(nlp)


In [ ]:
romeo_and_juliet.blobify()


In [ ]:
romeo_and_juliet.get_analysis()


In [ ]:
romeo_and_juliet.chapter_analysis()
books.append(romeo_and_juliet)

### Alice's Adventures in Wonderland

In [ ]:
BookImporter.get_book(11, 'Alice’s Adventures in Wonderland')
with open('data\Alice’s_Adventures_in_Wonderland.txt', 'r', encoding='UTF-8') as file:
    alice = Book('Alice in Wonderland', file.read())
chapter_markers = r'(CHAPTER I.\nDown the Rabbit-Hole)|(CHAPTER II.\nThe Pool of Tears)|(CHAPTER III.\nA Caucus-Race and a Long Tale)|(CHAPTER IV.\nThe Rabbit Sends in a Little Bill)|(CHAPTER V.\nAdvice from a Caterpillar)|(CHAPTER VI.\nPig and Pepper)|(CHAPTER VII.\nA Mad Tea-Party)|(CHAPTER VIII.\nThe Queen’s Croquet-Ground)|(CHAPTER IX.\nThe Mock Turtle’s Story)|(CHAPTER X.\nThe Lobster Quadrille)|(CHAPTER XI.\nWho Stole the Tarts\?)|(CHAPTER XII.\nAlice’s Evidence)'


In [ ]:
alice.split_into_chapters(chapter_markers)


In [ ]:
alice.do_nlp(nlp)


In [ ]:
alice.chapter_nlp(nlp)


In [ ]:
alice.blobify()


In [ ]:
alice.get_analysis()


In [ ]:
alice.chapter_analysis()
books.append(alice)

### The War of the Worlds

In [ ]:
BookImporter.get_book(36, 'War of the Worlds')
with open('data\War_of_the_Worlds.txt', 'r', encoding='UTF-8') as file:
    war_of_the_worlds = Book('War of the Worlds', file.read())

chapter_markers = r'\n[IVX]+\.\n'


In [ ]:
war_of_the_worlds.split_into_chapters(chapter_markers)


In [ ]:
war_of_the_worlds.do_nlp(nlp)


In [ ]:
war_of_the_worlds.chapter_nlp(nlp)


In [ ]:
war_of_the_worlds.blobify()


In [ ]:
war_of_the_worlds.get_analysis()


In [ ]:
war_of_the_worlds.chapter_analysis()
books.append(war_of_the_worlds)

### Wuthering Heights

In [ ]:
with open('data\Wuthering_Heights.txt', 'r', encoding='UTF-8') as file:
    wuthering_heights = Book('Wuthering Heights', file.read())

chapter_markers = r'CHAPTER [IVX]+'


In [ ]:
wuthering_heights.split_into_chapters(chapter_markers)


In [ ]:
wuthering_heights.do_nlp(nlp)


In [ ]:
wuthering_heights.chapter_nlp(nlp)


In [ ]:
wuthering_heights.blobify()


In [ ]:
wuthering_heights.get_analysis()


In [ ]:
wuthering_heights.chapter_analysis()
books.append(wuthering_heights)

### Pride and Prejudice

In [ ]:
with open('data\Pride_and_Prejudice.txt', 'r', encoding='UTF-8') as file:
    pride_prejudice = Book('Pride and Prejudice', file.read())

chapter_markers = r'(Chapter I\.\])|CHAPTER [IVXL]+\.'


In [ ]:
pride_prejudice.split_into_chapters(chapter_markers)

In [ ]:
pride_prejudice.do_nlp(nlp)

In [ ]:
pride_prejudice.chapter_nlp(nlp)

In [ ]:
pride_prejudice.get_analysis()


In [ ]:
pride_prejudice.chapter_analysis()
books.append(pride_prejudice)

### Moby Dick

In [ ]:
with open('data\Moby-Dick_or,_the_Whale.txt', 'r', encoding='UTF-8') as file:
    moby_dick = Book('Moby Dick', file.read())


In [ ]:
chapter_markers = r'(\nCHAPTER .+)|(\nEPILOGUE.)'

In [ ]:

moby_dick.split_into_chapters(chapter_markers)


In [ ]:
moby_dick.do_nlp(nlp, 10000)


In [ ]:
moby_dick.chapter_nlp(nlp)


In [ ]:
moby_dick.blobify()


In [ ]:
moby_dick.get_analysis()


In [ ]:
moby_dick.chapter_analysis()
books.append(moby_dick)

### Ulysses

In [ ]:
BookImporter.get_book(4300, 'Ulysses')
with open('data\\Ulysses.txt', 'r', encoding='UTF-8') as file:
    ulysses = Book('Ulysses', file.read())


In [ ]:
chapter_markers = r''
for i in range(1, 18):
    chapter_markers += r'(\n\[ {} \])|'.format(i)
chapter_markers += r'(\n\[ 18 \])'

In [ ]:

ulysses.split_into_chapters(chapter_markers)


In [ ]:
ulysses.do_nlp(nlp, 10000)


In [ ]:
ulysses.chapter_nlp(nlp)


In [ ]:
ulysses.blobify()


In [ ]:

ulysses.get_analysis()


In [ ]:
ulysses.chapter_analysis()
books.append(ulysses)

### A Christmas Carol

In [ ]:
with open('data\A_Christmas_Carol.txt', 'r', encoding='UTF-8') as file:
    christmas_carol = Book('A Christmas Carol', file.read())

In [ ]:
chapter_markers = r'STAVE (.+)'

In [ ]:
christmas_carol.split_into_chapters(chapter_markers)

In [ ]:
christmas_carol.do_nlp(nlp)

In [ ]:
christmas_carol.chapter_nlp(nlp)

In [ ]:
christmas_carol.blobify()

In [ ]:
christmas_carol.get_analysis()

In [ ]:
christmas_carol.chapter_analysis()
books.append(christmas_carol)

## Dorian Grey

In [ ]:
with open('data\The_Picture_of_Dorian_Gray.txt', 'r', encoding='UTF-8') as file:
    dorian_gray = Book('The Picture of Dorian Gray', file.read())


In [ ]:
chapter_markers = r'\n(CHAPTER .+)'

In [ ]:
dorian_gray.split_into_chapters(chapter_markers)

In [ ]:
dorian_gray.do_nlp(nlp)

In [ ]:
dorian_gray.chapter_nlp(nlp)

In [ ]:
dorian_gray.blobify()

In [ ]:
dorian_gray.get_analysis()

In [ ]:
dorian_gray.chapter_analysis()
books.append(dorian_gray)

## War and Peace

In [ ]:
with open('data\War_and_Peace.txt', 'r', encoding='UTF-8') as file:
    war_and_peace = Book('War and Peace', file.read())


In [ ]:
chapter_markers = r'\nCHAPTER [IXVC]+\n'

In [ ]:
war_and_peace.split_into_chapters(chapter_markers)

In [ ]:
war_and_peace.do_nlp(nlp)

In [ ]:
war_and_peace.chapter_nlp(nlp)

In [ ]:
war_and_peace.blobify()

In [ ]:
war_and_peace.get_analysis()

In [ ]:
war_and_peace.chapter_analysis()
books.append(war_and_peace)

## Conduct similarity analysis and display results

In [ ]:
# Do a similarity analysis between each book in the books list
checker = SimilarityChecker(books)
checker.calculate_all_similarities()


In [ ]:
# Show the results of the similarity analysis
checker.display_matrix()